Test notebook to practice reading s3 files to SageMaker. The role returned by get_execution_role() needs to be added to the s3 Bucket policy within the Principal Key as "AWS":"arn:aws:iam::144273247162:role/SageMakerExecutionRole"

Currently the bucket is also allowing public access, so I'm not sure if this will work if the data is made private. However, I ran into an error when trying to access the public bucket without adding this role to the Policy.

## Imports

In [1]:
import os
import pandas as pd
import boto3
import sagemaker
from sagemaker import get_execution_role
import tempfile
import io
import gzip

In [2]:
role = get_execution_role()
print(role)

region = boto3.Session().region_name
sess = sagemaker.Session()

arn:aws:iam::144273247162:role/SageMakerExecutionRole


In [8]:
bucket = 'cis5190-foodreviews'

s3_client = boto3.client('s3')
contents = s3_client.list_objects(Bucket=bucket)['Contents']

In [9]:
for f in contents:
    print(f['Key'])

GoogleNews-vectors-negative300.bin.gz
Reviews_cleanText_noSW.csv
Reviews_cleanText_sw.csv


In [5]:
file_path = 'GoogleNews-vectors-negative300.bin.gz'

In [22]:
obj = s3_client.get_object(Bucket=bucket, Key=file_path)

### For .csv files

In [ ]:
data = pd.read_csv(io.BytesIO(obj['Body'].read()))
data.to_csv('Reviews_cleaned_noSW.csv', index=False)

### For gzipped file read

In [11]:
s3 = boto3.resource('s3')
obj = s3.Object(bucket, file_path)
buf = io.BytesIO(obj.get()["Body"].read()) # reads whole gz file into memory
i = 0
for line in gzip.GzipFile(fileobj=buf):
    print(line)
    i += 1
    if i > 100:
        break

b'3000000 300\n'
b'</s> \x00\x00\x94:\x00\x00k\xba\x00\x00\xa79\x00\x00\xc9:\x00\x00\x91:\x00\x00\xb8\xba\x00\x00\x00\xb8\x00\x00\xdc\xb9\x00\x00\x17\xba\x00\x00\x8d:\x00\x00\x86\xba\x00\x00"\xba\x00\x00F\xba\x00\x00\xb8:\x00\x00\xd7\xba\x00\x00&\xba\x00\x00\xd6:\x00\x00\x84\xba\x00\x00\xa6\xba\x00\x00+:\x00\x00\xda\xb9\x00\x00\x8d\xba\x00\x00\xc8:\x00\x00\x90\xb9\x00\x00\x139\x00\x00\xce:\x00\x00\xb2:\x00\x00Z\xba\x00\x00\xb8\xba\x00\x00\xcf:\x00\x00\x859\x00\x00@\xba\x00\x00\xdd\xb8\x00\x00\x99\xba\x00\x00\xcf:\x00\x00,:\x00\x00-\xba\x00\x00D6\x00\x00\x94:\x00\x00\xe09\x00\x00\xc2\xb9\x00\x00\x97\xba\x00\x00\xa6:\x00\x00l\xb6\x00\x00\x8b9\x00\x00\xd3\xb9\x00\x00\x149\x00\x00\r\xb8\x00\x00G:\x00\x00\x08\xba\x00\x00\x95\xb8\x00\x00\x1e:\x00\x00\x05\xba\x00\x00\xd5\xba\x00\x00\xe5\xb9\x00\x00\x82\xba\x00\x00\xa0\xba\x00\x00\xa9\xb9\x00\x00\x908\x00\x00\x99\xba\x00\x00\x06\xba\x00\x00\xb9:\x00\x00\xb09\x00\x00\xf9\xb9\x00\x00\xb4\xba\x00\x00\xc09\x00\x00\xbd\xba\x00\x00\x1f\xba\x00\x00R: